In [1]:
import csv
import os
import pymc as pm
from pymc import do, observe
import pandas as pd
import numpy as np
import arviz as az
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from pytensor import tensor as pt
import pickle as pkl
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import itertools as it
import country_converter as cc
import math
import random

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
data = pd.read_csv("../data/ortiz-bobea/data2/regdata_preferred_case.csv")
data_len = len(data.fd_tmean)

# Ortiz-bobea regression formula: 
# fd_log_tfp ~ fd_tmean + fd_tmean_sq + fd_prcp + fd_prcp_sq

# Build model

In [3]:
# year and country fixed effect coefficient matrices
year_mult_mat = [np.zeros(data_len) for year in set(data.year)]
country_mult_mat = [np.zeros(data_len) for country in set(data.ISO3)]
country_index = -1
curr_country = ""
for row_index, row in enumerate(data.itertuples()):
    if row.ISO3 != curr_country:
        country_index += 1
        curr_country = row.ISO3
    year_index = row.year - 1962
    country_mult_mat[country_index][row_index] = 1
    year_mult_mat[year_index][row_index] = 1

In [9]:
with pm.Model() as model:

    fd_temp_prior = pm.Normal("fd_temp_prior", np.mean(data.fd_tmean), np.std(data.fd_tmean))
    fd_temp_std = pm.HalfNormal("fd_temp_std", 10)
    # fd_temp_posterior = pm.Normal("fd_temp_posterior", fd_temp_prior, fd_temp_std, shape=(data_len))
    fd_temp_posterior = pm.Normal("fd_temp_posterior", fd_temp_prior, fd_temp_std, observed=data.fd_tmean)

    fd_sq_temp_prior = pm.Normal("fd_sq_temp_prior", np.mean(data.fd_tmean_sq), np.std(data.fd_tmean_sq))
    fd_sq_temp_std = pm.HalfNormal("fd_sq_temp_std", 10)
    # fd_sq_temp_posterior = pm.Normal("fd_sq_temp_posterior", fd_sq_temp_prior, fd_sq_temp_std, shape=(data_len))
    fd_sq_temp_posterior = pm.Normal("fd_sq_temp_posterior", fd_sq_temp_prior, fd_sq_temp_std, observed=data.fd_tmean_sq)

    fd_precip_prior = pm.Normal("fd_precip_prior", np.mean(data.fd_prcp), np.std(data.fd_prcp))
    fd_precip_std = pm.HalfNormal("fd_precip_std", 10)
    # fd_precip_posterior = pm.Normal("fd_precip_posterior", fd_precip_prior, fd_precip_std, shape=(data_len))
    fd_precip_posterior = pm.Normal("fd_precip_posterior", fd_precip_prior, fd_precip_std, observed=data.fd_prcp)

    fd_sq_precip_prior = pm.Normal("fd_sq_precip_prior", np.mean(data.fd_prcp_sq), np.std(data.fd_prcp_sq))
    fd_sq_precip_std = pm.HalfNormal("fd_sq_precip_std", 10)
    # fd_sq_precip_posterior = pm.Normal("fd_sq_precip_posterior", fd_sq_precip_prior, fd_sq_precip_std, shape=(data_len))
    fd_sq_precip_posterior = pm.Normal("fd_sq_precip_posterior", fd_sq_precip_prior, fd_sq_precip_std, observed=data.fd_prcp_sq)

    tfp_intercept = pm.Normal('tfp_intercept',0,10)
    fd_temp_tfp_coef = pm.Normal('fd_temp_tfp_coef',0,10)
    fd_sq_temp_tfp_coef = pm.Normal('fd_sq_temp_tfp_coef',0,10)
    fd_precip_tfp_coef = pm.Normal("fd_precip_tfp_coef",0,10)
    fd_sq_precip_tfp_coef = pm.Normal("fd_sq_precip_tfp_coef",0,10)

    year_coefs = pt.expand_dims(pm.Normal("year_coefs", 0, 10, shape=(len(set(data.year)))),axis=1)
    year_fixed_effects = pm.Deterministic("year_fixed_effects",pt.sum((year_coefs*year_mult_mat),axis=0))

    country_coefs = pt.expand_dims(pm.Normal("country_coefs", 0, 10, shape=(len(set(data.ISO3)))),axis=1)
    country_fixed_effects = pm.Deterministic("country_fixed_effects",pt.sum((country_coefs*country_mult_mat),axis=0))

    print(country_fixed_effects.eval().shape)
    print(year_fixed_effects.eval().shape)
    
    tfp_prior = pm.Deterministic(
        "tfp_prior",
        tfp_intercept +
        (fd_temp_tfp_coef * fd_temp_posterior) +
        (fd_sq_temp_tfp_coef * fd_sq_temp_posterior) +
        (fd_precip_tfp_coef * fd_precip_posterior) +
        (fd_sq_precip_tfp_coef * fd_sq_precip_posterior) +
        year_fixed_effects +
        country_fixed_effects
    )
    
    tfp_std = pm.HalfNormal('tfp_std', sigma=10)
    # tfp_posterior = pm.Normal('tfp_posterior', mu=tfp_prior, sigma=tfp_std, shape=(data_len))
    tfp_posterior = pm.Normal('tfp_posterior', mu=tfp_prior, sigma=tfp_std, observed=data.fd_log_tfp)

    # prior = pm.sample_prior_predictive()
    # trace = pm.sample()
    # posterior = pm.sample_posterior_predictive(trace, extend_inferencedata=True)

(9255,)
(9255,)
(9255,)
(9255,)


# Add observed data to model and sample

In [19]:
observed_model = observe(model, {
    "fd_temp_posterior":data.fd_tmean,
    "fd_sq_temp_posterior":data.fd_tmean_sq,
    "fd_precip_posterior":data.fd_prcp,
    "fd_sq_precip_posterior":data.fd_prcp_sq,
    "tfp_posterior":data.fd_log_tfp
})

with observed_model:
    prior = pm.sample_prior_predictive()
    trace = pm.sample()
    posterior = pm.sample_posterior_predictive(trace, extend_inferencedata=True)

with open ('../models/ortiz-bobea-reproduction-year-country-fixed-effects.pkl', 'wb') as buff:
    pkl.dump({
        "prior":prior,
        "trace":trace,
        "posterior":posterior
    },buff)

Sampling: [fd_precip_posterior, fd_precip_prior, fd_precip_std, fd_precip_tfp_coef, fd_sq_precip_posterior, fd_sq_precip_prior, fd_sq_precip_std, fd_sq_precip_tfp_coef, fd_sq_temp_posterior, fd_sq_temp_prior, fd_sq_temp_std, fd_sq_temp_tfp_coef, fd_temp_posterior, fd_temp_prior, fd_temp_std, fd_temp_tfp_coef, tfp_intercept, tfp_posterior, tfp_std]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [fd_temp_prior, fd_temp_std, fd_sq_temp_prior, fd_sq_temp_std, fd_precip_prior, fd_precip_std, fd_sq_precip_prior, fd_sq_precip_std, tfp_intercept, fd_temp_tfp_coef, fd_sq_temp_tfp_coef, fd_precip_tfp_coef, fd_sq_precip_tfp_coef, tfp_std]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 199 seconds.
Sampling: [fd_precip_posterior, fd_sq_precip_posterior, fd_sq_temp_posterior, fd_temp_posterior, tfp_posterior]


# Estimate historical anthropogenic effect on TFP

In [3]:
# Green months
green_months = {}
green_month_data = pd.read_csv("../data/ortiz-bobea/data2/ndvi_co/peak_bottom_ndvi_month_country.csv")
for row in green_month_data.itertuples():
    peak = row[5]
    season = [i for i in range(peak-2,peak+3)]
    for i in range(len(season)):
        if season[i] < 1:
            season[i] = season[i] + 12
        elif season[i] > 12:
            season[i] = season[i] - 12
    green_months[row.ISO3] = season

In [222]:
gcms = ["BCC-CSM2-MR","CanESM5","CNRM-CM6-1","HadGEM3-GC31-LL","IPSL-CM6A-LR","MIROC6","MRI-ESM2-0"]
gcm_data = {gcm:{"hist_temp":{},"hist_prcp":{},"hist_nat_temp":{},"hist_nat_prcp":{}} for gcm in gcms}

for gcm in gcms:
    print(gcm)
    try:
        hist_data = pd.read_csv(f"../data/ortiz-bobea/data2/CMIP6_co/historical_{gcm}_1948-2020_cropland.csv")
    except FileNotFoundError:
        hist_data = pd.read_csv(f"../data/ortiz-bobea/data2/CMIP6_co/historical_{gcm}_1950-2020_cropland.csv")
    try:
        hist_nat_data = pd.read_csv(f"../data/ortiz-bobea/data2/CMIP6_co/hist-nat_{gcm}_1948-2020_cropland.csv")
    except FileNotFoundError:
        hist_nat_data = pd.read_csv(f"../data/ortiz-bobea/data2/CMIP6_co/hist-nat_{gcm}_1950-2020_cropland.csv")

    hist_temp, hist_nat_temp, hist_prcp, hist_nat_prcp = {key:[] for key in list(set(data.ISO3))}, {key:[] for key in list(set(data.ISO3))}, {key:[] for key in list(set(data.ISO3))}, {key:[] for key in list(set(data.ISO3))}
    
    for year in range(1961,2021):
        for country in set(data.ISO3):
    
            hist_rows = hist_data.loc[(hist_data["year"] == year) & (hist_data["ISO3"] == country)]
            hist_rows = [row for row in hist_rows.itertuples() if row.month in green_months[country]]
        
            nat_rows = hist_nat_data.loc[(hist_nat_data["year"] == year) & (hist_nat_data["ISO3"] == country)]
            nat_rows = [row for row in nat_rows.itertuples() if row.month in green_months[country]]
            
            hist_temp[country].append(np.mean([((row.tasmax + row.tasmin)/2)-273.15 for row in hist_rows]))
            hist_prcp[country].append(np.sum([row.pr for row in hist_rows]))
        
            hist_nat_temp[country].append(np.mean([((row.tasmax + row.tasmin)/2)-273.15 for row in nat_rows]))
            hist_nat_prcp[country].append(np.sum([row.pr for row in nat_rows]))

    gcm_data[gcm]["hist_temp"] = hist_temp
    gcm_data[gcm]["hist_prcp"] = hist_prcp
    gcm_data[gcm]["hist_nat_temp"] = hist_nat_temp
    gcm_data[gcm]["hist_nat_prcp"] = hist_nat_prcp

with open ("../data/ortiz-bobea/data2/gcm_data.pkl", 'wb') as buff:
    pkl.dump(gcm_data,buff)

BCC-CSM2-MR
CanESM5
CNRM-CM6-1
HadGEM3-GC31-LL
IPSL-CM6A-LR
MIROC6
MRI-ESM2-0


In [4]:
trace = pd.read_pickle('../models/ortiz-bobea-reproduction-year-country-fixed-effects.pkl')["trace"]
gcm_data = pd.read_pickle("../data/ortiz-bobea/data2/gcm_data.pkl")

In [5]:
coef1 = trace.posterior.fd_temp_tfp_coef.data.flatten()
coef2 = trace.posterior.fd_sq_temp_tfp_coef.data.flatten()
coef3 = trace.posterior.fd_precip_tfp_coef.data.flatten()
coef4 = trace.posterior.fd_sq_precip_tfp_coef.data.flatten()
# coef1=[-7.774284e-03,-9.106163e-03,2.237827e-04,-1.279122e-02,-3.066498e-03,-9.331451e-03,-2.837678e-03,-1.034688e-02,-3.141956e-04,-1.008950e-02]
# coef2=[5.822780e-05,4.438732e-05,-1.736405e-04,1.227045e-04,-1.416188e-04,5.294643e-05,-2.175081e-04,1.405300e-04,-1.737980e-04,6.780401e-05]
# coef3=[2.626333e-04,1.473205e-04,2.151584e-04,1.722369e-04,2.235674e-04,2.109962e-04,2.151249e-04,2.049795e-04,1.544933e-04,1.784246e-04]
# coef4=[-2.704724e-07,-1.242315e-07,-2.438181e-07,-1.666162e-07,-2.420376e-07,-1.927169e-07,-2.217054e-07,-2.093278e-07,-1.670311e-07,-1.892745e-07]

In [6]:
# Revenue Weights
revenue_data = pd.read_csv("../data/ortiz-bobea/data/TFP_USDA/revenue_shares.csv")
country_weights = {}
for row in revenue_data.itertuples():
    if row[3] in set(data.ISO3):
        country_weights[row[3]] = np.mean([row[5],row[6],row[7],row[8],row[9],row[10]])
weight_sum = sum(list(country_weights.values()))
for country, val in country_weights.items():
    country_weights[country] = val/weight_sum

In [7]:
# Historical climate means
country_climate_means = {}
country_1961_means = {}
ccm_file = pd.read_csv("../data/ortiz-bobea/data2/country_climate_means.csv")
for row in ccm_file.itertuples():
    country_climate_means[row[2]] = {"mean_temp":row[3], "mean_prcp":row[4]}
c1961_file = pd.read_csv("../data/ortiz-bobea/data2/country_climate_1961.csv")
for row in c1961_file.itertuples():
    country_1961_means[row[2]] = {"mean_temp":row[6],"mean_prcp":row[4]}

In [8]:
temp_hist_dev, temp_nat_dev, prcp_hist_dev, prcp_nat_dev = {}, {}, {}, {}
t1, t2, p1, p2 = {}, {}, {}, {}
t_diff, t_2_diff, p_diff, p_2_diff, = {}, {}, {}, {}

for gcm in gcm_data:
    
    temp_hist_dev[gcm] = {}
    temp_nat_dev[gcm] = {}
    prcp_hist_dev[gcm] = {}
    prcp_nat_dev[gcm] = {}
    t1[gcm] = {}
    t2[gcm] = {}
    p1[gcm] = {}
    p2[gcm] = {}
    t_diff[gcm] = {}
    p_diff[gcm] = {}
    t_2_diff[gcm] = {}
    p_2_diff[gcm] = {}
    
    for country in set(data.ISO3):
    
        temp_hist_dev[gcm][country] = np.array(gcm_data[gcm]["hist_temp"][country]) - country_climate_means[country]["mean_temp"]
        temp_nat_dev[gcm][country] = np.array(gcm_data[gcm]["hist_nat_temp"][country]) - country_climate_means[country]["mean_temp"]
        prcp_hist_dev[gcm][country] = 1 + (np.array(gcm_data[gcm]["hist_prcp"][country]) - country_climate_means[country]["mean_prcp"]) / country_climate_means[country]["mean_prcp"]
        prcp_nat_dev[gcm][country] = 1 + (np.array(gcm_data[gcm]["hist_nat_prcp"][country]) - country_climate_means[country]["mean_prcp"]) / country_climate_means[country]["mean_prcp"]
        
        t1[gcm][country] = temp_nat_dev[gcm][country] + country_1961_means[country]["mean_temp"]
        t2[gcm][country] = temp_hist_dev[gcm][country] + country_1961_means[country]["mean_temp"]
        p1[gcm][country] = prcp_nat_dev[gcm][country] * country_1961_means[country]["mean_prcp"]
        p2[gcm][country] = prcp_hist_dev[gcm][country] * country_1961_means[country]["mean_prcp"]
    
        t_diff[gcm][country] = t2[gcm][country] - t1[gcm][country]
        p_diff[gcm][country] = p2[gcm][country] - p1[gcm][country]
        t_2_diff[gcm][country] = pow(t2[gcm][country],2) - pow(t1[gcm][country],2)
        p_2_diff[gcm][country] = pow(p2[gcm][country],2) - pow(p1[gcm][country],2)

In [16]:
random.seed(0)
gcm_sample = random.choices(list(gcm_data.keys()), k=len(coef1))
global_impacts = []
country_res = {}
for country in set(data.ISO3):
    country_res[country] = []
    for i in range(len(coef1)):
        country_res[country].append(
            t_diff[gcm_sample[i]][country]*coef1[i] + 
            t_2_diff[gcm_sample[i]][country]*coef2[i] + 
            p_diff[gcm_sample[i]][country]*coef3[i] + 
            p_2_diff[gcm_sample[i]][country]*coef4[i]
            )
gcm_global_impacts = []
for coef in range(len(coef1)):
    coef_vals = []
    for year in range(0,60):
        year_vals = []
        for country, values in country_res.items():
            year_vals.append(values[coef][year] * country_weights[country])
        coef_vals.append(np.sum(year_vals))
    gcm_global_impacts.append(np.cumsum(coef_vals))
global_impacts.append([arr[-1] for arr in gcm_global_impacts])

print(np.mean(global_impacts),np.std(global_impacts))

-0.23769493458472346 0.131996054362902
